In [1]:
import numpy as np
import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.layers import BatchNormalization,Activation
import keras.backend as K
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
labels=pd.read_csv('cifar10Labels.csv')

In [3]:
model = applications.VGG16(weights='imagenet',include_top=False,input_shape=(128,128,3))

In [4]:
for layers in model.layers:
    layers.trainable=False

In [5]:
from keras.models import Sequential
top_model=Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(2048))
top_model.add(BatchNormalization())
top_model.add(Activation('relu'))

In [6]:
# top_model.add(Dense(512))
# top_model.add(BatchNormalization())
# top_model.add(Activation('relu'))
top_model.add(Dense(10))
top_model.add(BatchNormalization())
top_model.add(Activation('softmax'))

In [7]:
model =Model(inputs=model.input,outputs=top_model(model.output))

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
train_dir='cifar10_train'
valid_dir='cifar10_valid'

In [10]:
filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5'
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
mcCallBack = keras.callbacks.ModelCheckpoint(filepath,save_best_only=True,mode='auto')
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)
decaylearn = keras.callbacks.ReduceLROnPlateau(patience=10)

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=32,target_size=(128,128),class_mode='categorical')
test_generator=test_datagen.flow_from_directory(valid_dir,batch_size=32,target_size=(128,128),class_mode='categorical')

Found 49500 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [ ]:
model.fit_generator(train_generator,
                    callbacks=[tbCallBack,mcCallBack,earlystop,decaylearn],
                    epochs=5,
                    validation_data=test_generator)

Epoch 1/5
1547/1547 [==============================] - 4280s 3s/step - loss: 1.0552 - acc: 0.6621 - val_loss: 0.9178 - val_acc: 0.6960
Epoch 2/5
1547/1547 [==============================] - 4250s 3s/step - loss: 0.8349 - acc: 0.7228 - val_loss: 0.7856 - val_acc: 0.7300
Epoch 3/5
1547/1547 [==============================] - 4252s 3s/step - loss: 0.7528 - acc: 0.7469 - val_loss: 0.7816 - val_acc: 0.7280
Epoch 4/5
 521/1547 [=========>....................] - ETA: 46:28 - loss: 0.7045 - acc: 0.7608